In [10]:
import pandas as pd
import numpy as np
from statsmodels.sandbox.regression.gmm import GMM
from scipy.optimize import minimize

In [2]:
# Load the dataset
data = pd.read_csv("/MBAN- Schulich/MBAN- Sem 2/Predictive Modeling/Mid term Project/midterm_partone.csv")

In [3]:
# Display the first few rows to understand its structure
print(data.head())

   Constant  Stock Change  Inventory Turnover  Operating Profit  \
0         1      0.870332            1.795946          0.115846   
1         1     -0.047347            1.395501          0.436967   
2         1      0.001176            1.664563          0.541016   
3         1     -0.901200            1.605738          0.539399   
4         1     -0.176353            1.591451          0.539938   

   Interaction Effect  Current Ratio  Quick Ratio  Debt Asset Ratio  
0            0.208053       1.672527     0.255171          0.473317  
1            0.609788       1.637261     0.221763          0.489967  
2            0.900555       1.640619     0.189141          0.374269  
3            0.866133       1.436221     0.131944          0.224399  
4            0.859285       1.433140     0.183095          0.213446  


In [9]:
# Define dependent, independent, and instrumental variables
y_vals = np.array(data["Stock Change"])  # Dependent variable
x_vals = np.array(data[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])  # Endogenous variables
iv_vals = np.array(data[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])  # Instrumental variables

In [11]:
# Define a custom GMM class to handle the moment conditions with delta
class GMMWithBias:
    def __init__(self, y_vals, x_vals, iv_vals):
        self.y_vals = y_vals
        self.x_vals = x_vals
        self.iv_vals = iv_vals

    # Define the moment conditions with delta as an additional bias term
    def moment_conditions(self, params):
        # Unpack the parameters (p0, p1, p2, p3 are coefficients; delta is the bias term)
        p0, p1, p2, p3, delta = params
        # Calculate residuals including delta term
        residuals = self.y_vals - (p0 + p1 * self.x_vals[:, 0] + p2 * self.x_vals[:, 1] + p3 * self.x_vals[:, 2]) - delta
        
        # Define the moment conditions
        g = np.column_stack((residuals, 
                             residuals * self.x_vals[:, 0], 
                             residuals * self.x_vals[:, 1], 
                             residuals * self.x_vals[:, 2], 
                             residuals * self.iv_vals[:, 0], 
                             residuals * self.iv_vals[:, 1], 
                             residuals * self.iv_vals[:, 2]))
        return g

    # Define the GMM objective function as the sum of squared moments
    def gmm_objective(self, params):
        moments = self.moment_conditions(params)
        return np.sum(moments ** 2)

In [12]:
# Initialize parameters: coefficients for x_vals plus delta
initial_params = np.array([0.1, 0.1, 0.1, 0.1, 0.1])  # The last element is delta

In [13]:
# Instantiate the GMM model
gmm_model = GMMWithBias(y_vals, x_vals, iv_vals)

In [14]:
# Optimize to find the best-fit parameters
result = minimize(gmm_model.gmm_objective, initial_params)

In [15]:
# Extract and print the estimated parameters
B_estimated = result.x[:-1]  # Estimated coefficients for x_vals
delta_estimated = result.x[-1]  # Estimated bias term delta

In [16]:
# Display the results
print("Estimated Coefficients (B):", B_estimated)
print("Estimated Bias Term (Delta):", delta_estimated)
print("Optimization Success:", result.success)
print("Objective Function Value:", result.fun)

Estimated Coefficients (B): [-0.02226153  0.00066278 -0.03505035  0.00049336]
Estimated Bias Term (Delta): -0.022261532077181536
Optimization Success: False
Objective Function Value: 1250539.0814367211
